In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
import warnings
import itertools
warnings.filterwarnings("ignore")
import arviz as az
import scipy
import matplotlib.patches as patches
import statsmodels.stats.api as sms
import scipy.stats.kde as kde

# Read 16S data

In [2]:
df_16S = pd.read_excel('./16S_relative_abundance.xlsx', index_col=0)
df_16S.columns = [c.replace('/','_slash_').replace(' ','_space_').replace('[','_leftsquarebracket_').replace(']','_rightsquarebracket_').replace('-','_dash_').replace('.','_dot_').replace('(','_leftroundbracket').replace(')','_rightroundbracket_') for c in df_16S.columns]
df_16S.head()

,Abiotrophia_defectiva,Abiotrophia_sp_dot__HMSC24B09,Absiella_dolichum,Acetanaerobacterium_elongatum,Acetitomaculum_ruminis,Acetivibrio_cellulolyticus,Acetivibrio_ethanolgignens,Acetoanaerobium_sticklandii,Acetobacter_nitrogenifigens,Acetobacter_papayae,...,_leftsquarebracket_Eubacterium_rightsquarebracket__combesii,_leftsquarebracket_Eubacterium_rightsquarebracket__eligens,_leftsquarebracket_Eubacterium_rightsquarebracket__hallii,_leftsquarebracket_Eubacterium_rightsquarebracket__siraeum,_leftsquarebracket_Eubacterium_rightsquarebracket__yurii,_leftsquarebracket_Haemophilus_rightsquarebracket__ducreyi,_leftsquarebracket_Hallella_rightsquarebracket__seregens,_leftsquarebracket_Propionibacterium_rightsquarebracket__namnetense,_leftsquarebracket_Ruminococcus_rightsquarebracket__gnavus,_leftsquarebracket_Ruminococcus_rightsquarebracket__torques
SampleID,,,,,,,,,,,,,,,,,,,,,
G212_SS001685,0.0,0.000009,0.000018,0.0,0.000000,0.0,0.000097,0.0,0.0,0.0,...,0.0,0.013440,0.012065,0.000423,0.0,0.000000,0.000000,0.0,0.001914,0.005027
G212_SS001781,0.0,0.000000,0.000201,0.0,0.000000,0.0,0.000046,0.0,0.0,0.0,...,0.0,0.004616,0.020981,0.000221,0.0,0.000016,0.000000,0.0,0.002235,0.010386
G212_SS001782,0.0,0.000003,0.000018,0.0,0.000000,0.0,0.000036,0.0,0.0,0.0,...,0.0,0.000522,0.037412,0.000629,0.0,0.000000,0.000000,0.0,0.003169,0.008648
G212_SS001783,0.0,0.000000,0.000027,0.0,0.000000,0.0,0.000042,0.0,0.0,0.0,...,0.0,0.001219,0.047010,0.001616,0.0,0.000000,0.000000,0.0,0.003124,0.012215
G212_SS001784,0.0,0.000006,0.000611,0.0,0.000003,0.0,0.000047,0.0,0.0,0.0,...,0.0,0.000855,0.023347,0.000116,0.0,0.000000,0.000006,0.0,0.002769,0.019837


# Add pseudocount for zeros

In [3]:
for sample_id in df_16S.index:
    sample = np.array(df_16S.loc[sample_id])
    minval = np.min(sample[np.nonzero(sample)]) # minimum non-zero value
    sample[sample==0] = minval
    df_16S.loc[sample_id] = sample
df_16S.head()

,Abiotrophia_defectiva,Abiotrophia_sp_dot__HMSC24B09,Absiella_dolichum,Acetanaerobacterium_elongatum,Acetitomaculum_ruminis,Acetivibrio_cellulolyticus,Acetivibrio_ethanolgignens,Acetoanaerobium_sticklandii,Acetobacter_nitrogenifigens,Acetobacter_papayae,...,_leftsquarebracket_Eubacterium_rightsquarebracket__combesii,_leftsquarebracket_Eubacterium_rightsquarebracket__eligens,_leftsquarebracket_Eubacterium_rightsquarebracket__hallii,_leftsquarebracket_Eubacterium_rightsquarebracket__siraeum,_leftsquarebracket_Eubacterium_rightsquarebracket__yurii,_leftsquarebracket_Haemophilus_rightsquarebracket__ducreyi,_leftsquarebracket_Hallella_rightsquarebracket__seregens,_leftsquarebracket_Propionibacterium_rightsquarebracket__namnetense,_leftsquarebracket_Ruminococcus_rightsquarebracket__gnavus,_leftsquarebracket_Ruminococcus_rightsquarebracket__torques
SampleID,,,,,,,,,,,,,,,,,,,,,
G212_SS001685,0.000009,0.000009,0.000018,0.000009,0.000009,0.000009,0.000097,0.000009,0.000009,0.000009,...,0.000009,0.013440,0.012065,0.000423,0.000009,0.000009,0.000009,0.000009,0.001914,0.005027
G212_SS001781,0.000002,0.000002,0.000201,0.000002,0.000002,0.000002,0.000046,0.000002,0.000002,0.000002,...,0.000002,0.004616,0.020981,0.000221,0.000002,0.000016,0.000002,0.000002,0.002235,0.010386
G212_SS001782,0.000003,0.000003,0.000018,0.000003,0.000003,0.000003,0.000036,0.000003,0.000003,0.000003,...,0.000003,0.000522,0.037412,0.000629,0.000003,0.000003,0.000003,0.000003,0.003169,0.008648
G212_SS001783,0.000004,0.000004,0.000027,0.000004,0.000004,0.000004,0.000042,0.000004,0.000004,0.000004,...,0.000004,0.001219,0.047010,0.001616,0.000004,0.000004,0.000004,0.000004,0.003124,0.012215
G212_SS001784,0.000003,0.000006,0.000611,0.000003,0.000003,0.000003,0.000047,0.000003,0.000003,0.000003,...,0.000003,0.000855,0.023347,0.000116,0.000003,0.000003,0.000006,0.000003,0.002769,0.019837


# Read metadata

In [28]:
df_meta = pd.read_excel('fos_metadata.xlsx', sheet_name='metadata').set_index('Sample Name')
df_meta = df_meta.rename({'study_day':'Day', 'dose_g':'Diet'}, axis=1)
df_meta = df_meta[['Subject','Day','Diet']]
df_meta = df_meta[df_meta.Subject.notnull()]

# remove subject that has single data
subject_to_remove = []
for sub in set(df_meta.Subject):
    if len(df_meta[df_meta.Subject==sub])==1:
        subject_to_remove.append(sub)
df_meta = df_meta[~df_meta.Subject.isin(subject_to_remove)]

df_meta.head()

,Subject,Day,Diet
Sample Name,,,
G212_SS001685,101-030,50.135417,0.0
G212_SS001781,101-007,50.052778,0.0
G212_SS001782,101-028,41.486111,0.0
G212_SS001783,101-028,44.583333,0.0
G212_SS001784,101-039,22.177083,2.5


# Select the 20 most abundant taxa

In [29]:
topX = 20
df_16S_T = df_16S.loc[df_meta.index].T
df_16S_T['mean'] = df_16S_T.mean(axis=1)
df_16S_T = df_16S_T.sort_values(by=['mean'],axis=0,ascending=False)
df_16S_T = df_16S_T.drop('mean', axis=1)
df_16S_topX = df_16S_T.iloc[0:topX].T
df_16S_topX.head()

,Lachnospiraceae,Faecalibacterium_prausnitzii,Dorea_longicatena,Fusicatenibacter_saccharivorans,Anaerostipes_hadrus,Bifidobacterium_adolescentis,Prevotella_copri,Blautia_obeum,_leftsquarebracket_Eubacterium_rightsquarebracket__hallii,Roseburia_faecis,Akkermansia_muciniphila,Bifidobacterium_longum,Coprococcus_comes,Blautia_wexlerae,Collinsella_aerofaciens,Gemmiger_formicilis,Ruminococcus_bicirculans,Bifidobacterium_bifidum,Roseburia_intestinalis,_leftsquarebracket_Ruminococcus_rightsquarebracket__torques
Sample Name,,,,,,,,,,,,,,,,,,,,
G212_SS001685,0.109473,0.244980,0.044713,0.023732,0.025135,0.000009,0.050613,0.013379,0.012065,0.109579,0.000009,0.000053,0.007787,0.008149,0.000009,0.036900,0.008722,0.000009,0.008828,0.005027
G212_SS001781,0.124860,0.061379,0.094882,0.086011,0.033223,0.113808,0.005373,0.028723,0.020981,0.000417,0.000002,0.018627,0.036220,0.015588,0.021449,0.009364,0.000025,0.000306,0.019442,0.010386
G212_SS001782,0.124927,0.195199,0.076336,0.044751,0.054878,0.032744,0.000003,0.032598,0.037412,0.005402,0.007822,0.012851,0.038855,0.019556,0.014065,0.016408,0.095361,0.000089,0.011295,0.008648
G212_SS001783,0.124150,0.217632,0.084494,0.027841,0.041561,0.033910,0.000004,0.024572,0.047010,0.000873,0.000865,0.010859,0.048427,0.011122,0.015233,0.032173,0.046831,0.000061,0.003185,0.012215
G212_SS001784,0.172932,0.119267,0.074408,0.044969,0.007456,0.005313,0.000003,0.021598,0.023347,0.025748,0.000025,0.012794,0.044210,0.020503,0.009367,0.002990,0.000008,0.000055,0.004381,0.019837


In [30]:
list(df_16S_topX.columns)

['Lachnospiraceae',
 'Faecalibacterium_prausnitzii',
 'Dorea_longicatena',
 'Fusicatenibacter_saccharivorans',
 'Anaerostipes_hadrus',
 'Bifidobacterium_adolescentis',
 'Prevotella_copri',
 'Blautia_obeum',
 '_leftsquarebracket_Eubacterium_rightsquarebracket__hallii',
 'Roseburia_faecis',
 'Akkermansia_muciniphila',
 'Bifidobacterium_longum',
 'Coprococcus_comes',
 'Blautia_wexlerae',
 'Collinsella_aerofaciens',
 'Gemmiger_formicilis',
 'Ruminococcus_bicirculans',
 'Bifidobacterium_bifidum',
 'Roseburia_intestinalis',
 '_leftsquarebracket_Ruminococcus_rightsquarebracket__torques']

# Normalize max value of 16S data to 1

In [31]:
bacterial_taxa = list(df_16S_topX.columns)
df_16S_topX_w_meta = pd.merge(df_meta, df_16S_topX/df_16S_topX.max().max(), left_index=True, right_index=True, how='inner')
df_16S_topX_w_meta.head()

,Subject,Day,Diet,Lachnospiraceae,Faecalibacterium_prausnitzii,Dorea_longicatena,Fusicatenibacter_saccharivorans,Anaerostipes_hadrus,Bifidobacterium_adolescentis,Prevotella_copri,...,Akkermansia_muciniphila,Bifidobacterium_longum,Coprococcus_comes,Blautia_wexlerae,Collinsella_aerofaciens,Gemmiger_formicilis,Ruminococcus_bicirculans,Bifidobacterium_bifidum,Roseburia_intestinalis,_leftsquarebracket_Ruminococcus_rightsquarebracket__torques
Sample Name,,,,,,,,,,,,,,,,,,,,,
G212_SS001685,101-030,50.135417,0.0,0.160302,0.358727,0.065474,0.034752,0.036805,0.000013,0.074114,...,0.000013,0.000077,0.011403,0.011933,0.000013,0.054032,0.012772,0.000013,0.012927,0.007361
G212_SS001781,101-007,50.052778,0.0,0.182834,0.089877,0.138937,0.125947,0.048649,0.166651,0.007868,...,0.000003,0.027276,0.053037,0.022825,0.031407,0.013712,0.000037,0.000447,0.028469,0.015208
G212_SS001782,101-028,41.486111,0.0,0.182932,0.285832,0.111779,0.065530,0.080359,0.047947,0.000004,...,0.011454,0.018818,0.056896,0.028636,0.020595,0.024026,0.139638,0.000131,0.016540,0.012663
G212_SS001783,101-028,44.583333,0.0,0.181794,0.318681,0.123725,0.040768,0.060858,0.049655,0.000006,...,0.001266,0.015901,0.070912,0.016286,0.022306,0.047111,0.068575,0.000089,0.004664,0.017887
G212_SS001784,101-039,22.177083,2.5,0.253227,0.174644,0.108956,0.065849,0.010918,0.007781,0.000004,...,0.000036,0.018735,0.064737,0.030023,0.013717,0.004378,0.000012,0.000081,0.006415,0.029048


# Calculate log-derivatives (dlog(B)/dt)

In [32]:
df_deriv = deepcopy(df_16S_topX_w_meta)
for curr_sub in set(df_deriv.Subject):
    curr_df = df_deriv[df_deriv.Subject==curr_sub].sort_values(by='Day')
    for taxon in bacterial_taxa:
        xdata = np.array(curr_df['Day'])
        ydata = np.array(curr_df[taxon])
        cs = CubicSpline(xdata, ydata)
        csd1 = cs.derivative(nu=1)
        ydata_d1 = csd1(xdata)
        df_deriv.loc[df_deriv.Subject==curr_sub, taxon] = ydata_d1
df_deriv.head()

,Subject,Day,Diet,Lachnospiraceae,Faecalibacterium_prausnitzii,Dorea_longicatena,Fusicatenibacter_saccharivorans,Anaerostipes_hadrus,Bifidobacterium_adolescentis,Prevotella_copri,...,Akkermansia_muciniphila,Bifidobacterium_longum,Coprococcus_comes,Blautia_wexlerae,Collinsella_aerofaciens,Gemmiger_formicilis,Ruminococcus_bicirculans,Bifidobacterium_bifidum,Roseburia_intestinalis,_leftsquarebracket_Ruminococcus_rightsquarebracket__torques
Sample Name,,,,,,,,,,,,,,,,,,,,,
G212_SS001685,101-030,50.135417,0.0,0.079955,-0.305886,0.102930,-0.013917,0.105432,-0.000003,3.156090e-03,...,0.000007,-0.000045,0.004126,0.010843,-0.000032,-0.078115,0.000114,-0.000018,0.027831,0.015173
G212_SS001781,101-007,50.052778,0.0,-0.350997,0.228537,-0.160566,-0.106509,-0.013696,-0.119875,1.004450e+00,...,0.000008,-0.053520,-0.060096,-0.019855,-0.048225,-0.001002,-0.000118,-0.001719,0.149186,-0.015777
G212_SS001782,101-028,41.486111,0.0,-0.033039,0.213033,0.028208,0.046750,-0.001887,0.008359,1.272122e-08,...,-0.009206,0.007687,0.011981,0.013598,0.010594,0.019558,-0.049944,-0.000054,0.010307,-0.007841
G212_SS001783,101-028,44.583333,0.0,-0.006915,0.040284,0.028467,0.013208,0.012539,-0.012373,8.850795e-07,...,-0.005774,0.000416,0.010895,0.007495,0.000309,0.001751,-0.068225,-0.000055,0.026606,0.006785
G212_SS001784,101-039,22.177083,2.5,0.322493,-0.026923,0.136809,-0.006970,-0.040000,0.003048,-5.017792e-06,...,-0.000190,0.047185,0.071169,0.001243,0.006736,-0.003201,-0.000011,0.000102,-0.003610,-0.104799


# Construct regression matrix

In [34]:
# Ymat should be samples by bacteria
Ymat = df_deriv[bacterial_taxa].values
Ymat = Ymat.flatten(order='F')
Ymat = StandardScaler().fit_transform(Ymat.reshape(-1,1)).reshape(1,-1)[0] # standardize
Ymat

array([ 0.89394064, -3.99639251, -0.38828353, ..., -0.01298858,
        0.0542411 ,  1.18126258])

In [35]:
len(Ymat)

18920

In [36]:
Xmat = np.zeros(shape=(topX*len(df_deriv.index), (topX+2)*topX))
for k in np.arange(topX):
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)] = 1
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+1] = df_deriv.Diet.values
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+2:(k+1)*(topX+2)] = df_16S_topX_w_meta[bacterial_taxa].values
Xmat

array([[1.00000000e+00, 0.00000000e+00, 1.60302274e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 1.82833722e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 1.82931605e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.36232233e-02, 1.59073939e-03, 1.96240747e-03],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.33919309e-06, 4.68198934e-03, 1.49485202e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        8.91873007e-06, 5.12113480e-02, 5.09259487e-02]])

# Write stan program

## write data

In [39]:
json_str = '{\n"N" : %d,\n'%(len(Ymat))

# y variable
json_str += '\"dlogX\" : [%s],\n'%(','.join(list(Ymat.astype(str))))

# x variable
for k1,c1 in enumerate(bacterial_taxa):
    # growth rate
    json_str += '\"growth_rate_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)].astype(str))))
    # diet response
    json_str += '\"rs_response_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)+1].astype(str))))
    # bacterial interactions
    for k2,c2 in enumerate(bacterial_taxa):
        v = list(Xmat[:,k1*(topX+2)+2+k2].astype(str))
        json_str += '\"pairwise_interaction_%s_%s\" : [%s]'%(c1,c2,','.join(v))
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            json_str += '\n}'
        else:
            json_str += ',\n'
#print(json_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.data.json", "w")
text_file.write("%s" % json_str)
text_file.close()

## write model

In [40]:
# data block
model_str = 'data {\n'
model_str += '\tint<lower=0> N;\n'
model_str += '\tvector[N] dlogX;\n'
for c1 in bacterial_taxa:
    model_str += '\tvector[N] growth_rate_%s;\n'%(c1)
    model_str += '\tvector[N] rs_response_%s;\n'%(c1)
    for c2 in bacterial_taxa:
        model_str += '\tvector[N] pairwise_interaction_%s_%s;\n'%(c1,c2)
model_str += '}\n'

# parameter block
model_str += 'parameters {\n\treal<lower=0,upper=1> sigma;\n'
for c1 in bacterial_taxa:
    model_str += '\treal alpha__%s;\n'%(c1) # growth rate
    model_str += '\treal epsilon__%s;\n'%(c1) # rs response
    for c2 in bacterial_taxa:
        model_str += '\treal beta__%s_%s;\n'%(c1,c2)
model_str += '}\n'       
        
# model block
model_str += 'model {\n\tsigma ~ uniform(0,1);\n'
for c1 in bacterial_taxa:
    model_str += '\talpha__%s ~ normal(0,1);\n'%(c1) # growth rate
    model_str += '\tepsilon__%s ~ normal(0,1);\n'%(c1) # rs response
    for c2 in bacterial_taxa:
        model_str += '\tbeta__%s_%s ~ normal(0,1);\n'%(c1,c2)
model_str += '\tdlogX ~ normal('
for c1 in bacterial_taxa:
    model_str += 'alpha__%s*growth_rate_%s+'%(c1,c1) # growth rate
    model_str += 'epsilon__%s*rs_response_%s+'%(c1,c1) # rs response
    for c2 in bacterial_taxa:
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s'%(c1,c2,c1,c2)
        else:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s+'%(c1,c2,c1,c2)
model_str += ', sigma);\n}'
#print(model_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa/mice_scfa.stan", "w")
text_file.write("%s" % model_str)
text_file.close()

# Plot stan output

In [70]:
def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
        x_hpd = x[(x > value[0]) & (x < value[1])]
        y_hpd = y[(x > value[0]) & (x < value[1])]
        modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes

In [71]:
fit = az.from_cmdstan(["output_%d.csv"%(i) for i in np.arange(1,4)])

In [72]:
lines = []
    
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
# inulin response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['rs_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])

# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.head()

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Bacteroides,-0.42,0.50,0.040,0.159749,False
1,basal_growth_rate,Faecalibacterium,-0.60,0.32,-0.140,-0.605190,False
2,basal_growth_rate,Lachnospiraceae,-0.66,0.26,-0.200,-0.856932,False
3,basal_growth_rate,gut_dash_metagenome,-0.14,0.78,0.320,1.414249,False
4,basal_growth_rate,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_das...,-0.34,0.57,0.115,0.486693,False


In [73]:
df_stan_output_summary.to_excel('bayesian_regression_summary_top20.xlsx')

# Plot

In [74]:
df_stan_output_summary = pd.read_excel('bayesian_regression_summary_top20.xlsx', index_col=0)
df_stan_output_summary = df_stan_output_summary[df_stan_output_summary.Significant==True]
df_stan_output_summary

,Type,Taxa,Left,Right,Middle,SNR,Significant
22,rs_response,Lachnospiraceae,0.11,0.70,0.405,2.684754,True
24,rs_response,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_das...,-0.64,-0.05,-0.345,-2.313357,True
29,rs_response,Anaerostipes,0.10,0.70,0.400,2.642284,True
31,rs_response,Bifidobacterium,0.19,0.78,0.485,3.229673,True
40,pairwise_interaction,"('Bacteroides', 'Bacteroides')",0.54,2.00,1.270,3.432835,True
41,pairwise_interaction,"('Bacteroides', 'Faecalibacterium')",-7.65,-4.91,-6.280,-9.042536,True
43,pairwise_interaction,"('Bacteroides', 'gut_dash_metagenome')",0.02,3.46,1.740,1.989138,True
44,pairwise_interaction,"('Bacteroides', 'Ruminococcus_dash_sp_dot__das...",0.51,4.05,2.280,2.536649,True
46,pairwise_interaction,"('Bacteroides', 'Blautia')",-3.70,-0.08,-1.890,-2.059112,True
61,pairwise_interaction,"('Faecalibacterium', 'Faecalibacterium')",1.53,4.27,2.900,4.187724,True
